In [25]:
import matplotlib.pyplot as plt
import pandas as pd
import healpy
from Handler import *
import os

In [26]:
# path_data = "/Volumes/elmichelangelo_external_ssd_1/Data/gaNdalF_paper_catalogs"
path_data = "/Users/P.Gebhardt/Development/PhD/data"
path_data_master_cat = "/Volumes/elmichelangelo_external_ssd_1/Data"
filename_flw_balrog = "2024-10-02_10-37_balrog_flw_Test_sample.pkl"
filename_flw_gandalf = "2024-10-02_10-37_gandalf_flw_Test_sample.pkl"
filename_master_cat = "Y3_mastercat_02_05_21.h5"
path_save_plots = "/Users/P.Gebhardt/Development/PhD/output/gaNdalF_paper"

In [27]:
df_balrog_flw = pd.read_pickle(f"{path_data}/{filename_flw_balrog}")
df_gandalf_flw = pd.read_pickle(f"{path_data}/{filename_flw_gandalf}")

In [28]:
columns = [
    "unsheared/mag_r",
    "unsheared/mag_i",
    "unsheared/mag_z",
    "Color unsheared MAG r-i",
    "Color unsheared MAG i-z",
    "unsheared/snr",
    "unsheared/size_ratio",
    "unsheared/weight",
    "unsheared/T"
]

In [29]:
def apply_cuts(data_frame, path_master_cat):
    """"""
    data_frame = unsheared_object_cuts(data_frame=data_frame)
    data_frame = flag_cuts(data_frame=data_frame)
    data_frame = unsheared_shear_cuts(data_frame=data_frame)
    data_frame = binary_cut(data_frame=data_frame)
    data_frame = mask_cut_healpy(
        data_frame=data_frame,
        master=path_master_cat
    )
    data_frame = unsheared_mag_cut(data_frame=data_frame)
    return data_frame

In [30]:
def replace_nan(data_frame, cols, default_values):
    """
    """
    for idx, col in enumerate(cols):
        data_frame[col] = data_frame[col].fillna(default_values[idx])
    return data_frame

In [31]:
def max_default(data_frame, cols, max_lim):
    for idx, col in enumerate(cols):
        arr_flw = data_frame[col].to_numpy()
        for i, val in enumerate(arr_flw):
            if val > max_lim[idx]:
                arr_flw[i] = max_lim[idx]
        data_frame[col] = arr_flw
    return data_frame

In [32]:
def min_default(data_frame, cols, min_lim):
    for idx, col in enumerate(cols):
        arr_flw = data_frame[col].to_numpy()
        for i, val in enumerate(arr_flw):
            if val < min_lim[idx]:
                arr_flw[i] = min_lim[idx]
        data_frame[col] = arr_flw
    return data_frame

In [33]:
lst_col_nan = [
    "unsheared/mag_r",
    "unsheared/mag_i",
    "unsheared/mag_z",
    "unsheared/snr",
    "unsheared/size_ratio",
    "unsheared/weight",
    "unsheared/T",
]

lst_nan = [
    df_balrog_flw["unsheared/mag_r"].max(), 
    df_balrog_flw["unsheared/mag_i"].max(),
    df_balrog_flw["unsheared/mag_z"].max(),
    0.01,
    df_balrog_flw["unsheared/size_ratio"].max(),
    df_balrog_flw["unsheared/weight"].max(),
    df_balrog_flw["unsheared/T"].max(),
]
lst_col_max = [
    "unsheared/mag_r",
    "unsheared/mag_i",
    "unsheared/mag_z",
    # "unsheared/weight",
    # "unsheared/T"
]

lst_max = [
    df_balrog_flw["unsheared/mag_r"].max(), 
    df_balrog_flw["unsheared/mag_i"].max(),
    df_balrog_flw["unsheared/mag_z"].max(),
    # df_balrog_flw["unsheared/weight"].max(),
    # df_balrog_flw["unsheared/T"].max()
]
lst_col_min = [
    "unsheared/snr"
]

lst_min = [
    df_balrog_flw["unsheared/snr"].min()
]

In [34]:
for col in ["unsheared/snr", "unsheared/size_ratio", "unsheared/weight", "unsheared/T"]:
    print("Balrog", col,  df_balrog_flw[col].min(), df_balrog_flw[col].max())
    print("gandalf", col,  df_gandalf_flw[col].min(), df_gandalf_flw[col].max(), len(df_gandalf_flw[df_gandalf_flw[col] < df_balrog_flw[col].min()]), len(df_gandalf_flw[df_gandalf_flw[col] > df_balrog_flw[col].max()]))

Balrog unsheared/snr 0.2922319918893921 2156553.400698355
gandalf unsheared/snr -1.3403619464477559 892543939.6454452 182 41
Balrog unsheared/size_ratio -0.863650170709954 4346136.564595506
gandalf unsheared/size_ratio -1.056310170403342 5958409.006839054 4 1
Balrog unsheared/weight 10.300535620007455 77.58102207323857
gandalf unsheared/weight 1.3665886212677467 5841394.611151484 987493 26272
Balrog unsheared/T -0.35429057220318616 1430981.5103585045
gandalf unsheared/T -1.6248105766957441 1483.259239868674 1211 0


In [35]:
df_gandalf_flw = replace_nan(df_gandalf_flw, lst_col_nan, lst_nan)

In [36]:
df_gandalf_flw["Color unsheared MAG r-i"] = df_gandalf_flw["unsheared/mag_r"] - df_gandalf_flw["unsheared/mag_i"]
df_gandalf_flw["Color unsheared MAG i-z"] = df_gandalf_flw["unsheared/mag_i"] - df_gandalf_flw["unsheared/mag_z"]

In [37]:
df_gandalf_flw["unsheared/flux_r"] = mag2flux(df_gandalf_flw["unsheared/mag_r"])
df_gandalf_flw["unsheared/flux_i"] = mag2flux(df_gandalf_flw["unsheared/mag_i"])
df_gandalf_flw["unsheared/flux_z"] = mag2flux(df_gandalf_flw["unsheared/mag_z"])

In [38]:
df_balrog_flw["unsheared/flux_r"] = mag2flux(df_balrog_flw["unsheared/mag_r"])
df_balrog_flw["unsheared/flux_i"] = mag2flux(df_balrog_flw["unsheared/mag_i"])
df_balrog_flw["unsheared/flux_z"] = mag2flux(df_balrog_flw["unsheared/mag_z"])

In [39]:
df_gandalf_flw_cut = df_gandalf_flw.copy()
df_gandalf_flw_cut = apply_cuts(df_gandalf_flw_cut, f"{path_data_master_cat}/{filename_master_cat}")

Apply unsheared object cuts
Length of catalog after applying unsheared object cuts: 7189227
Apply flag cuts
Length of catalog after applying flag cuts: 6248605
Apply unsheared shear cuts
Length of catalog after applying unsheared shear cuts: 2518616
perform binaries cut


/Users/P.Gebhardt/Development/PhD/GalaxyFlow/venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


len w/ binaries 2518331
define mask
pass:  2412196
fail:  106135
Apply unsheared mag cuts
Length of catalog after applying unsheared mag cuts: 2337548


In [40]:
df_balrog_flw_cut = df_balrog_flw.copy()
df_balrog_flw_cut = apply_cuts(df_balrog_flw_cut, f"{path_data_master_cat}/{filename_master_cat}")

Apply unsheared object cuts
Length of catalog after applying unsheared object cuts: 8557499
Apply flag cuts
Length of catalog after applying flag cuts: 7379787
Apply unsheared shear cuts
Length of catalog after applying unsheared shear cuts: 2593251
perform binaries cut
len w/ binaries 2592958
define mask
pass:  2480946
fail:  112012
Apply unsheared mag cuts
Length of catalog after applying unsheared mag cuts: 2415456


In [41]:
df_balrog_flw = df_balrog_flw[columns]
df_gandalf_flw = df_gandalf_flw[columns]

In [42]:
print(f"Length of Balrog objects: {len(df_balrog_flw)}")
print(f"Length of gaNdalF objects: {len(df_gandalf_flw)}")
print(f"Length of Balrog objects after mag cut: {len(df_balrog_flw_cut)}")
print(f"Length of gaNdalF objects after mag cut: {len(df_gandalf_flw_cut)}")

Length of Balrog objects: 8616255
Length of gaNdalF objects: 8614806
Length of Balrog objects after mag cut: 2415456
Length of gaNdalF objects after mag cut: 2337548


In [43]:
df_gandalf_flw_sub_cut = df_gandalf_flw_cut.sample(n=int(1E6))
df_balrog_flw_sub_cut = df_balrog_flw_cut.sample(n=int(1E6))

df_gandalf_flw_sub = df_gandalf_flw.sample(n=int(1E6))
df_balrog_flw_sub = df_balrog_flw.sample(n=int(1E6))

In [44]:
plot_compare_corner(
    data_frame_generated=df_gandalf_flw,
    data_frame_true=df_balrog_flw,
    dict_delta=None,
    epoch=None,
    title=f"Compare Measured Galaxy Properties Balrog-gaNdalF",
    columns=[
        "Color unsheared MAG r-i",
        "Color unsheared MAG i-z",
        "unsheared/mag_r",
        "unsheared/mag_i",
        "unsheared/mag_z",
        "unsheared/snr",
        "unsheared/size_ratio",
        "unsheared/weight",
        "unsheared/T"
    ],
    labels=[
        "r-i",
        "i-z",
        "mag r",
        "mag i",
        "mag z",
        "snr",
        "size ratio",
        "weight",
        "T"
    ],
    show_plot=False,
    save_plot=True,
    save_name=f"{path_save_plots}/compare_measured_galaxy_properties_datapoints.png",
    ranges=[
        [-0.5, 1.5],
        [-0.5, 1.5],
        [18, 24.5],
        [18, 24.5],
        [18, 24.5],
        [2, 100],
        [-0.5, 5],
        [10, 80],
        [0, 3.5]
    ]
)

(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
       

In [45]:
plot_compare_corner(
    data_frame_generated=df_gandalf_flw_cut,
    data_frame_true=df_balrog_flw_cut,
    dict_delta=None,
    epoch=None,
    title=f"Compare MCAL Measured Galaxy Properties Balrog-gaNdalF",
    columns=[
        "Color unsheared MAG r-i",
        "Color unsheared MAG i-z",
        "unsheared/mag_r",
        "unsheared/mag_i",
        "unsheared/mag_z",
        "unsheared/snr",
        "unsheared/size_ratio",
        "unsheared/weight",
        "unsheared/T"
    ],
    labels=[
        "r-i",
        "i-z",
        "mag r",
        "mag i",
        "mag z",
        "snr",
        "size ratio",
        "weight",
        "T"
    ],
    show_plot=False,
    save_plot=True,
    save_name=f"{path_save_plots}/compare_mcal_measured_galaxy_properties_datapoints.png",
    ranges=[
        [-0.5, 1.5],
        [-0.5, 1.5],
        [18, 24.5],
        [18, 24.5],
        [18, 24.5],
        [2, 100],
        [-0.5, 5],
        [10, 80],
        [0, 3.5]
    ]
)

(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
       

In [46]:
plot_balrog_histogram_with_error(
    df_gandalf=df_gandalf_flw,
    df_balrog=df_balrog_flw,
    columns=[
        "unsheared/mag_r",
        "unsheared/mag_i",
        "unsheared/mag_z",
        "Color unsheared MAG r-i",
        "Color unsheared MAG i-z",
        "unsheared/snr",
        "unsheared/size_ratio",
        "unsheared/weight",
        "unsheared/T"
    ],
    labels=[
        "mag r",
        "mag i",
        "mag z",
        "mag r-i",
        "mag i-z",
        "snr",
        "size ratio",
        "weight",
        "T"
    ],
    ranges=[
        [18, 24.5],  # mag r
        [18, 24.5],  # mag i
        [18, 24.5],  # mag z
        [-0.5, 1.5],  # mag r-i
        [-0.5, 1.5],  # mag i-z
        [2, 100],  # snr
        [-0.5, 5],  # size ratio
        [10, 80],  # weight
        [0, 3.5]  # T
    ],
    binwidths=[
        None,  # mag r
        None,  # mag i
        None,  # mag z
        0.08,  # mag r-i
        0.08,  # mag i-z
        2,  # snr
        0.2,  # size ratio
        2,  # weight
        0.2  # T
    ],
    title="Compare Histogram",
    show_plot=False,
    save_plot=True,
    save_name=f"{path_save_plots}/hist_plot.png"
)

In [47]:
plot_balrog_histogram_with_error(
    df_gandalf=df_gandalf_flw_cut,
    df_balrog=df_balrog_flw_cut,
    columns=[
        "unsheared/mag_r",
        "unsheared/mag_i",
        "unsheared/mag_z",
        "Color unsheared MAG r-i",
        "Color unsheared MAG i-z",
        "unsheared/snr",
        "unsheared/size_ratio",
        "unsheared/weight",
        "unsheared/T"
    ],
    labels=[
        "mag r",
        "mag i",
        "mag z",
        "mag r-i",
        "mag i-z",
        "snr",
        "size ratio",
        "weight",
        "T"
    ],
    ranges=[
        [18, 24.5],  # mag r
        [18, 24.5],  # mag i
        [18, 24.5],  # mag z
        [-0.5, 1.5],  # mag r-i
        [-0.5, 1.5],  # mag i-z
        [2, 100],  # snr
        [-0.5, 5],  # size ratio
        [10, 80],  # weight
        [0, 3.5]  # T
    ],
    binwidths=[
        None,  # mag r
        None,  # mag i
        None,  # mag z
        0.08,  # mag r-i
        0.08,  # mag i-z
        2,  # snr
        0.2,  # size ratio
        2,  # weight
        0.2  # T
    ],
    title="Compare MCAL Histogram",
    show_plot=False,
    save_plot=True,
    save_name=f"{path_save_plots}/mcal_hist_plot.png"
)